In [1]:
import os
from PIL import Image
import numpy as np
import tensorrt as trt
import gc
import torch
import matplotlib.pyplot as plt

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Image_size = 256
storage_dir = "./storage"
file_list = os.listdir(storage_dir+"/cache_"+str(Image_size))
print("Number of Images",len(file_list))
pics = [ storage_dir + "/cache_"+str(Image_size) + "/" +str(i) for i in file_list]

Number of Images 1050


In [3]:
np_pics = np.asarray(Image.open(pics[0]))
brain_tiles = np_pics.reshape(1,3,Image_size,Image_size)
for i in range(65):
    img = np.asarray(Image.open(pics[i+1]))
    if (img.shape == (256, 256, 3)):
        brain_tiles = np.concatenate((brain_tiles,img.reshape(1,3,Image_size,Image_size)),axis=0)
    

In [4]:
brain_tiles = torch.from_numpy(brain_tiles).float()

In [5]:
brain_tiles = brain_tiles[:1024,:3,:,:]

In [6]:
brain_tiles.shape

torch.Size([64, 3, 256, 256])

In [7]:
num_gpu = 2
chunk = brain_tiles.shape[0]//num_gpu

In [8]:
brain_batches = [brain_tiles[i:i+chunk,:,:,:] for i in range(0,brain_tiles.shape[0],chunk)]

In [9]:
len(brain_batches)

2

In [10]:
brain_batches[0].shape

torch.Size([32, 3, 256, 256])

In [11]:
brain_batches[1].shape

torch.Size([32, 3, 256, 256])

In [12]:
from concurrent.futures import ThreadPoolExecutor
import TensorRT.inference as inf
print(os.cpu_count())

6


In [13]:
import pycuda as cuda
import pycuda.autoinit

import nvidia_smi
nvidia_smi.nvmlInit()
from ctypes import cdll, c_char_p
libcudart = cdll.LoadLibrary('libcudart.so')
libcudart.cudaGetErrorString.restype = c_char_p
def cudaSetDevice(device_idx):
    ret = libcudart.cudaSetDevice(device_idx)
    if ret != 0:
        error_string = libcudart.cudaGetErrorString(ret)
        raise RuntimeError("cudaSetDevice: " + error_string)


In [14]:
def run_inf(context,h_input, d_input, h_output, d_output, stream,image):
    inf.load_images_to_buffer(images, h_input)
    cuda.memcpy_htod_async(d_input, h_input, stream)
    context.execute_v2(bindings=[int(d_input), int(d_output)])
    cuda.memcpy_dtoh_async(h_output, d_output, stream)
    
def run_batch_inf(engine,images,batch_sizes=8):
    h_input, d_input, h_output, d_output, stream = inf.allocate_buffers(engine,1,trt.float32)
    context = engine.create_execution_context()
    input_shape = (batch_sizes,3,256,256)
    context.set_binding_shape(0, input_shape)
    for i in range(0,images.shape[0],batch_size):
        run_inf(context,h_input, d_input, h_output, d_output, stream,images[i:i+batch_size,:,:,:])
        

In [15]:
TRT_LOGGER = trt.Logger(min_severity =trt.ILogger.INTERNAL_ERROR)
def load_engine(engine_file_path):
    assert os.path.exists(engine_file_path)
    print("Reading engine from file {}".format(engine_file_path))
    with open(engine_file_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        return runtime.deserialize_cuda_engine(f.read())


In [16]:
engines = []

In [17]:
engines

[]

In [18]:
len(brain_batches)

2

In [19]:
cudaSetDevice(0)
engines.append(load_engine("model.plan"))
cudaSetDevice(1)
engines.append(load_engine("model.plan"))

Reading engine from file model.plan
Reading engine from file model.plan


In [20]:
with ThreadPoolExecutor(2) as executor:
    results = executor.map(run_batch_inf,engines,brain_batches)

In [21]:
for r in results:
    print(r)

LogicError: cuMemHostAlloc failed: invalid device context

In [23]:
for r in results:
    print(type(r))


In [24]:
results

<generator object Executor.map.<locals>.result_iterator at 0x7f6bc32bb0b0>